---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

# Assignment 4 - Hypothesis Testing Using t-test
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
# I added this dictionary during the work on the last function to translate the way the quarters are represented
QUARTERS = {'q1': '-01Q', 'q2': '-04Q', 'q3': '-07Q', 'q4': '-10Q'}

In [4]:
# I am adding the constant alpha as the significance level
ALPHA = 0.01

In [5]:
#def find 

def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # I tried several options for file reading, including some kwargs, and this was the best.
    read_towns_temp = pd.read_csv('university_towns.txt', delimiter='\n') 
    # Since the function reset_index work on the row index, I had to transform
    read_towns_temp = read_towns_temp.T
    read_towns_temp.reset_index(inplace=True) # since Alabama came out as the Series index, I had to "push" it into the data
    read_towns_temp = read_towns_temp.T
    read_towns_temp.index = range(len(read_towns_temp)) # to assign a new 0,1,2... index
    
    # Now I will mark first the state rows, and match the relevant state to each towns
    read_towns_temp[1] = None # adding a None column for the state
    read_towns_temp.columns = ['City', 'State']
    for i in range(len(read_towns_temp)):
        if 'edit' in read_towns_temp.loc[i,'City']: # Since all the stste rows include 'edit', this helps updating the state colimn
            read_towns_temp.loc[i,'State'] = read_towns_temp.loc[i,'City']
        else:
            read_towns_temp.loc[i,'State'] = read_towns_temp.loc[i-1,'State']
             
    # We need to take the [edit] off
    read_towns_temp['State'] = read_towns_temp['State'].map(lambda x: x.strip('[edit]'))
    
    # Need to differentiate between a state and a city
    for i in range(len(read_towns_temp)):
        if 'edit' in read_towns_temp.loc[i,'City']:
            read_towns_temp.loc[i,'State(0)/City(1)'] = 0
        else:
            read_towns_temp.loc[i,'State(0)/City(1)'] = 1
    # Now I an keeping only the city rows
    read_towns = read_towns_temp[read_towns_temp['State(0)/City(1)'] == 1.0].copy()
    
    # Removing the paranthesis and what on the right
    #read_towns['City'] = read_towns['City'].apply(lambda x: x.split(' (')[0]) # original
    read_towns['City'] = read_towns.loc[:,'City'].str.split(' ', 0, expand=True)
    # Droping the state/City column
    read_towns = read_towns.drop('State(0)/City(1)', 1)
    read_towns = read_towns[['State', 'City']] # Change the order of the columns
    read_towns.index = range(len(read_towns)) # to assign a new 0,1,2... index
    return read_towns


get_list_of_university_towns()

,State,City
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [6]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    recession_temp = pd.read_excel('gdplev.xls', skiprows=5)
    recession_temp = recession_temp.iloc[2:,[4,6]] # Slicing relevant frame part1
    # Since I was asked to refer only to Q1 2000 onwards, I first identify the row number for that
    row_number_for_start = recession_temp[recession_temp['Quarter'] == '2000q1'].index[0]
    recession_temp = recession_temp.iloc[row_number_for_start:,:] # Slicing relevant frame part2
    
    recession_temp.index = range(len(recession_temp)) # to assign a new 0,1,2... index
    # The following loop create a new column, assigning '1' if two there's decline in two quarters
    recession_temp['q_in_recession'] = None
    for i in range(1, len(recession_temp)-1):
        # The if statement refers to all potential recession start - identifies decline in two Qs in a row
        if all([recession_temp.loc[i, 'GDP in billions of chained 2009 dollars.1'] < recession_temp.loc[i-1, 'GDP in billions of chained 2009 dollars.1'],\
               recession_temp.loc[i+1, 'GDP in billions of chained 2009 dollars.1'] < recession_temp.loc[i, 'GDP in billions of chained 2009 dollars.1']]):
            recession_temp.loc[i, 'q_in_recession'] = 1
        # The elif statement refers to quarters in a middle of recession, previous q is 1, and not yet end of recession
        elif recession_temp.loc[i-1, 'q_in_recession'] == 1 and not (recession_temp.loc[i, 'GDP in billions of chained 2009 dollars.1'] >\
                                                                        recession_temp.loc[i-1, 'GDP in billions of chained 2009 dollars.1'] and\
                                                                        recession_temp.loc[i-1, 'GDP in billions of chained 2009 dollars.1'] >\
                                                                        recession_temp.loc[i-2, 'GDP in billions of chained 2009 dollars.1']):
            recession_temp.loc[i, 'q_in_recession'] = 1
        else:
            recession_temp.loc[i, 'q_in_recession'] = 0
            
    #  The following loop creates a new column with the '1' in quarters where recession starts
    for i in range(1, len(recession_temp)):
        if recession_temp.loc[i-1, 'q_in_recession'] == 0 and \
        recession_temp.loc[i, 'q_in_recession'] == 1:
            recession_temp.loc[i, 'recession_start'] = 1
        else:
            recession_temp.loc[i, 'recession_start'] = 0
    # Now we pick the relevant quarter
    return recession_temp['Quarter'][recession_temp['recession_start'] == 1].values[0]
get_recession_start()

'2008q3'

In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    recession_temp = pd.read_excel('gdplev.xls', skiprows=5)
    recession_temp = recession_temp.iloc[2:,[4,6]] # Slicing relevant frame part1
    # Since I was asked to refer only to Q1 2000 onwards, I first identify the row number for that
    row_number_for_start = recession_temp[recession_temp['Quarter'] == '2000q1'].index[0]
    recession_temp = recession_temp.iloc[row_number_for_start:,:] # Slicing relevant frame part2
    
    recession_temp.index = range(len(recession_temp)) # to assign a new 0,1,2... index
    # The following loop create a new column, assigning '1' if two there's decline in two quarters
    recession_temp['q_in_recession'] = None
    for i in range(2, len(recession_temp)-1):
        # The if statement refers to all potential recession start - identifies decline in two Qs in a row
        if all([recession_temp.loc[i, 'GDP in billions of chained 2009 dollars.1'] < recession_temp.loc[i-1, 'GDP in billions of chained 2009 dollars.1'],\
               recession_temp.loc[i+1, 'GDP in billions of chained 2009 dollars.1'] < recession_temp.loc[i, 'GDP in billions of chained 2009 dollars.1']]):
            recession_temp.loc[i, 'q_in_recession'] = 1
        # The elif statement refers to quarters in a middle of recession, previous q is 1, and not yet end of recession
        elif recession_temp.loc[i-1, 'q_in_recession'] == 1 and not (recession_temp.loc[i, 'GDP in billions of chained 2009 dollars.1'] >\
                                                                        recession_temp.loc[i-1, 'GDP in billions of chained 2009 dollars.1'] and\
                                                                        recession_temp.loc[i-1, 'GDP in billions of chained 2009 dollars.1'] >\
                                                                        recession_temp.loc[i-2, 'GDP in billions of chained 2009 dollars.1']):
            recession_temp.loc[i, 'q_in_recession'] = 1
        else:
            recession_temp.loc[i, 'q_in_recession'] = 0
            
    #  The following loop creates a new column with the '1' in quarters where recession starts
    for i in range(1, len(recession_temp)):
        if recession_temp.loc[i-1, 'q_in_recession'] == 1 and \
        recession_temp.loc[i, 'q_in_recession'] == 0:
            recession_temp.loc[i, 'recession_start'] = 1
        else:
            recession_temp.loc[i, 'recession_start'] = 0
    # Now we pick the relevant quarter from collumn 'Quarter'
    return recession_temp['Quarter'][recession_temp['recession_start'] == 1].values[0]
get_recession_end()

'2009q4'

In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    # First 5 code lines rearrange the DF
    recession_temp = pd.read_excel('gdplev.xls', skiprows=5)
    recession_temp = recession_temp.iloc[2:,[4,6]] # Slicing relevant frame part1
    # Since I was asked to refer only to Q1 2000 onwards, I first identify the row number for that
    row_number_for_start = recession_temp[recession_temp['Quarter'] == '2000q1'].index[0]
    recession_temp = recession_temp.iloc[row_number_for_start:,:] # Slicing relevant frame part2
    
    recession_temp.index = range(len(recession_temp)) # to assign a new 0,1,2... index
    
    # let's create a column of real datetime:
    #for recession_temp['Real Quarter'] = 
    for i in range(1, len(recession_temp)):
        recession_temp.loc[i, 'Real Quarter'] = pd.Period(recession_temp.loc[i,'Quarter'],'Q')
    # Now slices only the recession quarters
    period_start = pd.Period(get_recession_start(), 'Q') # The month that starts the recession (Numeric)
    period_end = pd.Period(get_recession_end(), 'Q') # The month that ends the recession (Numeric)
    filtered = recession_temp[(recession_temp['Real Quarter'] >= period_start) &\
                              (recession_temp['Real Quarter'] < period_end)]
    min_gdp = filtered['Quarter'][filtered['GDP in billions of chained 2009 dollars.1'] == \
                                  filtered['GDP in billions of chained 2009 dollars.1'].min()].values[0]
    return min_gdp 

get_recession_bottom()

'2009q2'

In [9]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    housing = pd.read_csv("City_Zhvi_AllHomes.csv")
    # Taking out the irrelevant months
    housing = housing.drop(housing.loc[:,'1996-04':'1999-12'].head(0).columns, axis=1) 
    
    # The following loop creates new column each iteration, referring to last month of each quarter
    for third_column_idx in range(housing.columns.get_loc("2000-03"), len(housing.columns), 3):
        end_q_month = housing.columns[third_column_idx] # Starting month for each quarter
        stat_q_month = housing.columns[third_column_idx-2] # Ending month for each quarter 
        housing[stat_q_month + 'Q'] = housing[[stat_q_month, end_q_month]].mean(axis=1) #end_q_month + 'Q'
    # Now that I have the figures for each Q I can take out the rest of the months
    housing = housing.drop(housing.loc[:,'2000-01':'2016-08'].head(0).columns, axis=1) 
    housing['State Full'] = housing['State'].map(states)
    housing['State Full Region'] = housing['State Full'] + housing['RegionName']
    housing = housing.set_index(['State', 'RegionName']) # Setting multi-index in the shape of ["State","RegionName"]
    return housing


convert_housing_data_to_quarters()

,,RegionID,Metro,CountyName,SizeRank,2000-01Q,2000-04Q,2000-07Q,2000-10Q,2001-01Q,2001-04Q,...,2014-07Q,2014-10Q,2015-01Q,2015-04Q,2015-07Q,2015-10Q,2016-01Q,2016-04Q,State Full,State Full Region
State,RegionName,,,,,,,,,,,,,,,,,,,,,
NY,New York,6181,New York,Queens,1,NaN,NaN,NaN,NaN,NaN,NaN,...,523000.0,527900.0,532300.0,540900.0,557200.0,572450.0,583200.0,591200.0,New York,New YorkNew York
CA,Los Angeles,12447,Los Angeles-Long Beach-Anaheim,Los Angeles,2,207100.0,214450.0,220900.0,226200.0,233000.0,239100.0,...,509100.0,518850.0,528750.0,538150.0,547300.0,557500.0,566250.0,577300.0,California,CaliforniaLos Angeles
IL,Chicago,17426,Chicago,Cook,3,138450.0,143600.0,147850.0,152200.0,156900.0,161800.0,...,195850.0,201050.0,201350.0,205800.0,208400.0,207950.0,206200.0,208200.0,Illinois,IllinoisChicago
PA,Philadelphia,13271,Philadelphia,Philadelphia,4,52950.0,53600.0,54150.0,54750.0,55300.0,55550.0,...,115200.0,115700.0,116250.0,118000.0,121200.0,122150.0,123500.0,126900.0,Pennsylvania,PennsylvaniaPhiladelphia
AZ,Phoenix,40326,Phoenix,Maricopa,5,111900.0,114400.0,116050.0,117400.0,119550.0,121600.0,...,165450.0,168550.0,171550.0,174200.0,179050.0,183850.0,187850.0,191500.0,Arizona,ArizonaPhoenix
NV,Las Vegas,18959,Las Vegas,Clark,6,132600.0,134350.0,135400.0,137050.0,139550.0,141700.0,...,173350.0,175550.0,177550.0,181650.0,186800.0,190650.0,194500.0,197150.0,Nevada,NevadaLas Vegas
CA,San Diego,54296,San Diego,San Diego,7,222900.0,234350.0,245500.0,256050.0,267250.0,276350.0,...,489150.0,496450.0,503400.0,512050.0,519750.0,525350.0,529400.0,535900.0,California,CaliforniaSan Diego
TX,Dallas,38128,Dallas-Fort Worth,Dallas,8,84450.0,83900.0,84850.0,87800.0,89750.0,89300.0,...,108900.0,111500.0,113900.0,121050.0,128550.0,134600.0,140450.0,144650.0,Texas,TexasDallas
CA,San Jose,33839,San Jose,Santa Clara,9,374400.0,406300.0,431950.0,455250.0,470350.0,469800.0,...,697300.0,715100.0,731850.0,756650.0,776400.0,788850.0,803850.0,818850.0,California,CaliforniaSan Jose


In [11]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # First I am calling the university town function with the university towns, and preparing it for merging
    university_town_df = get_list_of_university_towns()
    university_town_df['StateCity'] = university_town_df['State'] + university_town_df['City'] # This column to have common wiht housing
    university_town_df['University'] = 'Yes' # This column to differentiate the university town in the merged DF
    
    # Housing_df will be the left DF in the merging
    housing_df = convert_housing_data_to_quarters()
    combined_data = pd.merge(housing_df, university_town_df, how='left', left_on ='State Full Region', \
                         right_on = 'StateCity', sort=False, indicator=True)
    period_start = get_recession_start()[:4] + QUARTERS[get_recession_start()[4:]] # changed the quarters headers to match the DF
    period_bottom = get_recession_bottom()[:4] + QUARTERS[get_recession_bottom()[4:]] # changed the quarters headers to match the DF 
    combined_data['Ratio'] = combined_data[period_start] / combined_data[period_bottom] # Calculating a ratio new column
    
    # Now I will cut the dataset to two groups: unitersity towns and others
    univ_data = combined_data[combined_data['University'] == 'Yes']
    print(univ_data['Ratio'].isnull().sum()) # Checking nulls
    univ_data = univ_data.dropna(subset=['Ratio']) # Removing nulls
    print(univ_data['Ratio'].isnull().sum()) # Checking nulls again

    non_univ_data = combined_data[combined_data['University'] != 'Yes']
    print(non_univ_data['Ratio'].isnull().sum()) # Checking nulls
    non_univ_data = non_univ_data.dropna(subset=['Ratio']) # Removing nulls
    print(non_univ_data['Ratio'].isnull().sum()) # Checking nulls again

    # t-test
    p_val = stats.ttest_ind(univ_data['Ratio'], non_univ_data['Ratio'])[1] 
    # Check the t-test against alpha
    if p_val < ALPHA:
        different = True
    else:
        different = False
    
    # Check which group is "better"
    univ_town_mean = univ_data['Ratio'].mean()
    non_univ_mean = non_univ_data['Ratio'].mean()
    
    if univ_town_mean < non_univ_mean:
        better = "university town"
    else:
        better = "non-university town"
    
    return (different, p_val, better)


run_ttest()

6
0
840
0


(False, 0.070057771506764926, 'university town')